## 1. Setup

In [ ]:
!pip install -q transformers datasets accelerate scikit-learn pandas tqdm

In [ ]:
import requests
import time
import json
from datetime import datetime, timedelta
from urllib.parse import urlparse
from collections import Counter

import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    roc_auc_score, accuracy_score, precision_score, recall_score,
    f1_score, confusion_matrix, precision_recall_curve, roc_curve
)
from datasets import Dataset as HFDataset
import matplotlib.pyplot as plt

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## 2. Configuration

Key changes from v1:
- **100k posts** (vs 60k) - more data helps
- **RoBERTa** (vs DistilBERT) - better semantic understanding
- **Domain-enhanced input** - "[github.com] Show HN: My Project"

In [ ]:
# === CONFIGURATION ===
POSTS_PER_CATEGORY = 35000      # 35k x 3 = 105k total posts
MONTHS_BACK = 24                # 2 years of data
HIT_THRESHOLD = 100             # 100+ points = success

MODEL_NAME = "roberta-base"     # Better than DistilBERT
MAX_LENGTH = 72                 # Slightly longer for domain prefix
BATCH_SIZE = 24                 # Fits in T4 GPU memory
EPOCHS = 4                      # More epochs with early stopping
LEARNING_RATE = 1.5e-5          # Slightly lower for RoBERTa

# Class weights to fix high false negative rate
# Analysis showed 25% FN vs 7% FP - model is too conservative
CLASS_WEIGHTS = torch.tensor([1.0, 1.5]).to(device)  # Boost positive class

print(f"Config: {POSTS_PER_CATEGORY*3:,} posts, {MODEL_NAME}, {EPOCHS} epochs")

## 3. Fetch Training Data

Fetching from Algolia HN API with improved domain tracking.

In [ ]:
def fetch_hn_posts(min_points, max_points, target_count, months_back=24):
    """Fetch HN posts from Algolia API."""
    base_url = "https://hn.algolia.com/api/v1/search_by_date"
    all_posts = []
    seen_ids = set()

    if max_points:
        points_filter = f"points>={min_points},points<={max_points}"
        desc = f"{min_points}-{max_points} pts"
    else:
        points_filter = f"points>={min_points}"
        desc = f"{min_points}+ pts"

    end_date = datetime.now()
    posts_per_month = max(target_count // months_back, 800)
    pbar = tqdm(range(months_back), desc=f"Fetching {desc}")

    for month_offset in pbar:
        month_end = end_date - timedelta(days=30 * month_offset)
        month_start = month_end - timedelta(days=30)

        params = {
            "tags": "story",
            "numericFilters": f"{points_filter},created_at_i>={int(month_start.timestamp())},created_at_i<={int(month_end.timestamp())}",
            "hitsPerPage": min(posts_per_month, 1000),
        }

        try:
            resp = requests.get(base_url, params=params, timeout=30)
            resp.raise_for_status()
            data = resp.json()

            for hit in data.get("hits", []):
                post_id = hit.get("objectID")
                if post_id and post_id not in seen_ids:
                    seen_ids.add(post_id)
                    url = hit.get("url", "")
                    if url:
                        try:
                            domain = urlparse(url).netloc.replace("www.", "")
                        except:
                            domain = ""
                    else:
                        domain = "self.hackernews"

                    all_posts.append({
                        "id": post_id,
                        "title": hit.get("title", ""),
                        "url": url,
                        "domain": domain,
                        "points": hit.get("points", 0),
                        "num_comments": hit.get("num_comments", 0),
                        "created_at": hit.get("created_at_i", 0),
                    })

            pbar.set_postfix({"total": len(all_posts)})
        except Exception as e:
            print(f"Error: {e}")

        time.sleep(0.3)
        if len(all_posts) >= target_count:
            break

    return all_posts[:target_count]

In [ ]:
print(f"Fetching {POSTS_PER_CATEGORY*3:,} posts (~8-12 minutes)...\n")

posts_high = fetch_hn_posts(100, None, POSTS_PER_CATEGORY, MONTHS_BACK)
posts_medium = fetch_hn_posts(20, 99, POSTS_PER_CATEGORY, MONTHS_BACK)
posts_low = fetch_hn_posts(1, 19, POSTS_PER_CATEGORY, MONTHS_BACK)

for p in posts_high: p['category'] = 'high'
for p in posts_medium: p['category'] = 'medium'
for p in posts_low: p['category'] = 'low'

df = pd.DataFrame(posts_high + posts_medium + posts_low)
print(f"\n✓ Fetched {len(df):,} posts")
print(f"  High (100+): {len(posts_high):,}")
print(f"  Medium (20-99): {len(posts_medium):,}")
print(f"  Low (1-19): {len(posts_low):,}")

## 4. Feature Engineering

Adding ALL recommended features:
1. **Domain prefix** - `[github.com] Title`
2. **Temporal features** - hour of day, day of week
3. **Title meta-features** - length, question mark, numbers, Show/Ask HN

In [ ]:
# Clean data
df = df[df['title'].notna() & (df['title'].str.len() > 5)].copy()
df['label'] = (df['points'] >= HIT_THRESHOLD).astype(int)

# === DOMAIN FEATURES ===
domain_counts = df['domain'].value_counts()
top_domains = set(domain_counts.head(50).index)

def get_domain_tag(domain):
    return domain if domain in top_domains else "other"

df['domain_tag'] = df['domain'].apply(get_domain_tag)

# === TEMPORAL FEATURES ===
# Convert timestamp to datetime
df['datetime'] = pd.to_datetime(df['created_at'], unit='s')
df['hour'] = df['datetime'].dt.hour
df['dayofweek'] = df['datetime'].dt.dayofweek  # 0=Monday, 6=Sunday

# Create time buckets (HN has optimal posting times)
def get_time_tag(hour, dow):
    # Peak times: US morning (6-10am PT = 14-18 UTC), weekdays
    is_weekday = dow < 5
    is_peak_hour = 14 <= hour <= 18
    if is_weekday and is_peak_hour:
        return "PEAK"
    elif is_weekday:
        return "WEEKDAY"
    else:
        return "WEEKEND"

df['time_tag'] = df.apply(lambda r: get_time_tag(r['hour'], r['dayofweek']), axis=1)

# === TITLE META-FEATURES ===
df['has_question'] = df['title'].str.contains(r'\?').astype(int)
df['has_number'] = df['title'].str.contains(r'\d').astype(int)
df['is_show_hn'] = df['title'].str.lower().str.startswith('show hn').astype(int)
df['is_ask_hn'] = df['title'].str.lower().str.startswith('ask hn').astype(int)
df['title_length'] = df['title'].str.len()

# Length bucket
def get_length_tag(length):
    if length < 40:
        return "SHORT"
    elif length < 70:
        return "MEDIUM"
    else:
        return "LONG"

df['length_tag'] = df['title_length'].apply(get_length_tag)

# === BUILD RICH INPUT TEXT ===
# Format: [domain] [TIME] [LEN] [?] Title
def build_input(row):
    parts = [f"[{row['domain_tag']}]"]
    parts.append(f"[{row['time_tag']}]")
    parts.append(f"[{row['length_tag']}]")
    if row['has_question']:
        parts.append("[Q]")
    if row['is_show_hn']:
        parts.append("[SHOW]")
    elif row['is_ask_hn']:
        parts.append("[ASK]")
    parts.append(row['title'])
    return " ".join(parts)

df['input_text'] = df.apply(build_input, axis=1)

print(f"Dataset: {len(df):,} posts, {df['label'].mean():.1%} hits")
print(f"\nExample inputs:")
for _, row in df.sample(3, random_state=42).iterrows():
    print(f"  {row['input_text'][:80]}...")

In [ ]:
# Feature statistics
print("Feature Impact Analysis:")
print("\nBy Time:")
for tag in ['PEAK', 'WEEKDAY', 'WEEKEND']:
    mask = df['time_tag'] == tag
    print(f"  {tag:10} {mask.sum():6,} posts, {df.loc[mask, 'label'].mean():.1%} hit rate")

print("\nBy Post Type:")
print(f"  Show HN:  {df['is_show_hn'].sum():6,} posts, {df.loc[df['is_show_hn']==1, 'label'].mean():.1%} hit rate")
print(f"  Ask HN:   {df['is_ask_hn'].sum():6,} posts, {df.loc[df['is_ask_hn']==1, 'label'].mean():.1%} hit rate")
print(f"  Question: {df['has_question'].sum():6,} posts, {df.loc[df['has_question']==1, 'label'].mean():.1%} hit rate")

print("\nBy Title Length:")
for tag in ['SHORT', 'MEDIUM', 'LONG']:
    mask = df['length_tag'] == tag
    print(f"  {tag:10} {mask.sum():6,} posts, {df.loc[mask, 'label'].mean():.1%} hit rate")

In [ ]:
# Train/val/test split
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['label'])

print(f"Train: {len(train_df):,} ({train_df['label'].mean():.1%} hits)")
print(f"Val:   {len(val_df):,} ({val_df['label'].mean():.1%} hits)")
print(f"Test:  {len(test_df):,} ({test_df['label'].mean():.1%} hits)")

## 5. Tokenize with RoBERTa

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_function(examples):
    return tokenizer(
        examples['input_text'],
        padding='max_length',
        truncation=True,
        max_length=MAX_LENGTH
    )

# Create HuggingFace datasets
train_dataset = HFDataset.from_pandas(train_df[['input_text', 'label']].reset_index(drop=True))
val_dataset = HFDataset.from_pandas(val_df[['input_text', 'label']].reset_index(drop=True))
test_dataset = HFDataset.from_pandas(test_df[['input_text', 'label']].reset_index(drop=True))

# Tokenize
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Set format for PyTorch
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

print(f"✓ Tokenized: train={len(train_dataset):,}, val={len(val_dataset):,}, test={len(test_dataset):,}")

## 6. Train with Class Weights

Using weighted cross-entropy to reduce false negatives (the main issue from v1).

In [ ]:
class WeightedTrainer(Trainer):
    """Custom trainer with class weights to reduce false negatives."""
    
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        
        loss_fct = nn.CrossEntropyLoss(weight=CLASS_WEIGHTS)
        loss = loss_fct(logits, labels)
        
        return (loss, outputs) if return_outputs else loss


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = torch.softmax(torch.tensor(logits), dim=-1)[:, 1].numpy()
    preds = (probs >= 0.5).astype(int)
    
    return {
        'roc_auc': roc_auc_score(labels, probs),
        'accuracy': accuracy_score(labels, preds),
        'precision': precision_score(labels, preds),
        'recall': recall_score(labels, preds),
        'f1': f1_score(labels, preds),
    }

In [ ]:
# Load model
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
model = model.to(device)
print(f"Model: {MODEL_NAME} ({sum(p.numel() for p in model.parameters())/1e6:.0f}M params)")

# Training arguments
training_args = TrainingArguments(
    output_dir='./checkpoints',
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE * 2,
    learning_rate=LEARNING_RATE,
    weight_decay=0.01,
    warmup_ratio=0.1,
    eval_strategy="steps",
    eval_steps=500,
    save_strategy="steps",
    save_steps=500,
    load_best_model_at_end=True,
    metric_for_best_model="roc_auc",
    greater_is_better=True,
    logging_steps=100,
    report_to="none",
    fp16=torch.cuda.is_available(),
    seed=42,
)

trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

print(f"\nTraining {EPOCHS} epochs (~25-35 min on T4)...")
trainer.train()

## 7. Evaluate and Calibrate

Apply temperature scaling to improve calibration (ECE was 0.115 in v1).

In [ ]:
# Get predictions on test set
model.eval()
test_loader = DataLoader(test_dataset, batch_size=64)

all_logits = []
all_labels = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Evaluating"):
        inputs = {k: v.to(device) for k, v in batch.items() if k != 'label'}
        labels = batch['label']
        outputs = model(**inputs)
        all_logits.append(outputs.logits.cpu())
        all_labels.append(labels)

logits = torch.cat(all_logits)
labels = torch.cat(all_labels).numpy()
probs_raw = torch.softmax(logits, dim=-1)[:, 1].numpy()

print(f"Test ROC AUC (raw): {roc_auc_score(labels, probs_raw):.4f}")

In [ ]:
# Temperature scaling for better calibration
class TemperatureScaler(nn.Module):
    def __init__(self):
        super().__init__()
        self.temperature = nn.Parameter(torch.ones(1))
    
    def forward(self, logits):
        return logits / self.temperature

# Optimize temperature on validation set
val_loader = DataLoader(val_dataset, batch_size=64)
val_logits = []
val_labels = []

with torch.no_grad():
    for batch in val_loader:
        inputs = {k: v.to(device) for k, v in batch.items() if k != 'label'}
        outputs = model(**inputs)
        val_logits.append(outputs.logits.cpu())
        val_labels.append(batch['label'])

val_logits = torch.cat(val_logits)
val_labels = torch.cat(val_labels)

# Find optimal temperature
temp_scaler = TemperatureScaler()
optimizer = torch.optim.LBFGS([temp_scaler.temperature], lr=0.01, max_iter=50)

def eval_temp():
    optimizer.zero_grad()
    scaled = temp_scaler(val_logits)
    loss = nn.CrossEntropyLoss()(scaled, val_labels)
    loss.backward()
    return loss

optimizer.step(eval_temp)
optimal_temp = temp_scaler.temperature.item()
print(f"Optimal temperature: {optimal_temp:.3f}")

# Apply temperature scaling to test predictions
scaled_logits = logits / optimal_temp
probs_calibrated = torch.softmax(scaled_logits, dim=-1)[:, 1].numpy()

print(f"Test ROC AUC (calibrated): {roc_auc_score(labels, probs_calibrated):.4f}")

### TF-IDF Ensemble

Train a simple TF-IDF + Logistic Regression model and ensemble with RoBERTa.
This captures keyword patterns that transformers might miss.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# Train TF-IDF model on titles only (no special tokens)
print("Training TF-IDF ensemble model...")
tfidf = TfidfVectorizer(max_features=10000, ngram_range=(1, 2), min_df=3)
X_train_tfidf = tfidf.fit_transform(train_df['title'])
X_test_tfidf = tfidf.transform(test_df['title'])

tfidf_model = LogisticRegression(max_iter=1000, class_weight='balanced', C=1.0)
tfidf_model.fit(X_train_tfidf, train_df['label'])

probs_tfidf = tfidf_model.predict_proba(X_test_tfidf)[:, 1]
tfidf_auc = roc_auc_score(labels, probs_tfidf)
print(f"TF-IDF ROC AUC: {tfidf_auc:.4f}")

# Ensemble: weighted average (RoBERTa gets higher weight)
ROBERTA_WEIGHT = 0.7
TFIDF_WEIGHT = 0.3

probs_ensemble = ROBERTA_WEIGHT * probs_calibrated + TFIDF_WEIGHT * probs_tfidf
ensemble_auc = roc_auc_score(labels, probs_ensemble)

print(f"\nEnsemble ROC AUC: {ensemble_auc:.4f}")
print(f"  Improvement over RoBERTa alone: {ensemble_auc - roc_auc_score(labels, probs_calibrated):+.4f}")

In [ ]:
# Use ensemble predictions for final model
probs_final = probs_ensemble  # Switch to ensemble

# Find optimal threshold
thresholds = np.arange(0.2, 0.8, 0.02)
best_f1 = 0
best_thresh = 0.5
results = []

for t in thresholds:
    preds = (probs_final >= t).astype(int)
    p = precision_score(labels, preds)
    r = recall_score(labels, preds)
    f1 = f1_score(labels, preds)
    results.append({'threshold': t, 'precision': p, 'recall': r, 'f1': f1})
    if f1 > best_f1:
        best_f1 = f1
        best_thresh = t

print(f"Optimal threshold (ensemble): {best_thresh:.2f}")
print(f"  Precision: {precision_score(labels, (probs_final >= best_thresh)):.3f}")
print(f"  Recall:    {recall_score(labels, (probs_final >= best_thresh)):.3f}")
print(f"  F1:        {best_f1:.3f}")

## 8. Full Evaluation Report

In [ ]:
preds_optimal = (probs_final >= best_thresh).astype(int)
cm = confusion_matrix(labels, preds_optimal)

# Calculate ECE (Expected Calibration Error)
n_bins = 10
bin_boundaries = np.linspace(0, 1, n_bins + 1)
bin_indices = np.digitize(probs_final, bin_boundaries[1:-1])

ece = 0
for i in range(n_bins):
    mask = bin_indices == i
    if mask.sum() > 0:
        bin_acc = labels[mask].mean()
        bin_conf = probs_final[mask].mean()
        ece += mask.sum() * abs(bin_acc - bin_conf)
ece /= len(labels)

print("=" * 60)
print("FINAL EVALUATION (v2 Ensemble Model)")
print("=" * 60)
print(f"\n{'Metric':<25} {'RoBERTa':>10} {'Ensemble':>10}")
print("-" * 47)
print(f"{'ROC AUC':<25} {roc_auc_score(labels, probs_calibrated):>10.4f} {roc_auc_score(labels, probs_final):>10.4f}")
print(f"{'Accuracy':<25} {'-':>10} {accuracy_score(labels, preds_optimal):>10.4f}")
print(f"{'Precision':<25} {'-':>10} {precision_score(labels, preds_optimal):>10.4f}")
print(f"{'Recall':<25} {'-':>10} {recall_score(labels, preds_optimal):>10.4f}")
print(f"{'F1 Score':<25} {'-':>10} {f1_score(labels, preds_optimal):>10.4f}")
print(f"\nCalibration (ECE):    {ece:.4f} {'✓' if ece < 0.1 else '✗'}")
print(f"Optimal Threshold:    {best_thresh:.2f}")
print(f"Temperature:          {optimal_temp:.3f}")
print(f"Ensemble Weights:     RoBERTa={ROBERTA_WEIGHT}, TF-IDF={TFIDF_WEIGHT}")

print(f"\nConfusion Matrix (threshold={best_thresh:.2f}):")
print(f"  TN={cm[0,0]:5,}  FP={cm[0,1]:5,}")
print(f"  FN={cm[1,0]:5,}  TP={cm[1,1]:5,}")

fn_rate = cm[1,0] / (cm[1,0] + cm[1,1])
fp_rate = cm[0,1] / (cm[0,0] + cm[0,1])
print(f"\n  False Negative Rate: {fn_rate:.1%} (target: <20%)")
print(f"  False Positive Rate: {fp_rate:.1%}")

## 9. Visualizations

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# 1. ROC Curve - Compare RoBERTa vs Ensemble
ax = axes[0, 0]
fpr1, tpr1, _ = roc_curve(labels, probs_calibrated)
fpr2, tpr2, _ = roc_curve(labels, probs_final)
ax.plot(fpr1, tpr1, 'b--', linewidth=1.5, alpha=0.7, label=f'RoBERTa ({roc_auc_score(labels, probs_calibrated):.3f})')
ax.plot(fpr2, tpr2, 'g-', linewidth=2, label=f'Ensemble ({roc_auc_score(labels, probs_final):.3f})')
ax.plot([0, 1], [0, 1], 'k--', alpha=0.3)
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title('ROC Curve Comparison')
ax.legend()
ax.grid(alpha=0.3)

# 2. Precision-Recall Curve
ax = axes[0, 1]
precision_curve, recall_curve, _ = precision_recall_curve(labels, probs_final)
ax.plot(recall_curve, precision_curve, 'g-', linewidth=2)
ax.axhline(labels.mean(), color='gray', linestyle='--', label=f'Baseline ({labels.mean():.2f})')
ax.set_xlabel('Recall')
ax.set_ylabel('Precision')
ax.set_title('Precision-Recall Curve (Ensemble)')
ax.legend()
ax.grid(alpha=0.3)

# 3. Score Distribution
ax = axes[0, 2]
ax.hist(probs_final[labels==0], bins=50, alpha=0.6, label='Not Hit', density=True, color='red')
ax.hist(probs_final[labels==1], bins=50, alpha=0.6, label='Hit', density=True, color='green')
ax.axvline(best_thresh, color='black', linestyle='--', linewidth=2, label=f'Threshold={best_thresh:.2f}')
ax.set_xlabel('Predicted Probability')
ax.set_ylabel('Density')
ax.set_title('Score Distribution (Ensemble)')
ax.legend()

# 4. Calibration Plot
ax = axes[1, 0]
bin_edges = np.linspace(0, 1, 11)
bin_indices_plot = np.digitize(probs_final, bin_edges[1:-1])
cal_actual = [labels[bin_indices_plot == i].mean() if (bin_indices_plot == i).sum() > 0 else np.nan for i in range(10)]
cal_pred = [probs_final[bin_indices_plot == i].mean() if (bin_indices_plot == i).sum() > 0 else np.nan for i in range(10)]
ax.plot([0, 1], [0, 1], 'k--', alpha=0.5, label='Perfect')
ax.scatter(cal_pred, cal_actual, s=100, zorder=5)
ax.plot(cal_pred, cal_actual, 'o-', label=f'Ensemble (ECE={ece:.3f})')
ax.set_xlabel('Mean Predicted')
ax.set_ylabel('Actual Hit Rate')
ax.set_title('Calibration Plot')
ax.legend()
ax.grid(alpha=0.3)

# 5. Metrics vs Threshold
ax = axes[1, 1]
results_df = pd.DataFrame(results)
ax.plot(results_df['threshold'], results_df['precision'], label='Precision', linewidth=2)
ax.plot(results_df['threshold'], results_df['recall'], label='Recall', linewidth=2)
ax.plot(results_df['threshold'], results_df['f1'], label='F1', linewidth=2)
ax.axvline(best_thresh, color='gray', linestyle=':', label=f'Optimal={best_thresh:.2f}')
ax.set_xlabel('Threshold')
ax.set_ylabel('Score')
ax.set_title('Metrics vs Threshold')
ax.legend()
ax.grid(alpha=0.3)

# 6. Confusion Matrix Heatmap
ax = axes[1, 2]
im = ax.imshow(cm, cmap='Blues')
ax.set_xticks([0, 1])
ax.set_yticks([0, 1])
ax.set_xticklabels(['Not Hit', 'Hit'])
ax.set_yticklabels(['Not Hit', 'Hit'])
ax.set_xlabel('Predicted')
ax.set_ylabel('Actual')
ax.set_title(f'Confusion Matrix (t={best_thresh:.2f})')
for i in range(2):
    for j in range(2):
        ax.text(j, i, f'{cm[i,j]:,}', ha='center', va='center', fontsize=14,
                color='white' if cm[i,j] > cm.max()/2 else 'black')

plt.tight_layout()
plt.savefig('evaluation_v2.png', dpi=150)
plt.show()
print("Saved: evaluation_v2.png")

## 10. Demo Predictions

In [ ]:
def predict_v2_ensemble(titles, domains=None, time_tags=None):
    """Predict with full feature set and ensemble."""
    if domains is None:
        domains = ["other"] * len(titles)
    if time_tags is None:
        time_tags = ["WEEKDAY"] * len(titles)  # Default assumption
    
    # Build rich input text
    inputs_list = []
    for title, domain, time_tag in zip(titles, domains, time_tags):
        domain_tag = domain if domain in top_domains else "other"
        length_tag = "SHORT" if len(title) < 40 else ("MEDIUM" if len(title) < 70 else "LONG")
        has_q = "[Q]" if "?" in title else ""
        is_show = "[SHOW]" if title.lower().startswith("show hn") else ""
        is_ask = "[ASK]" if title.lower().startswith("ask hn") else ""
        
        parts = [f"[{domain_tag}]", f"[{time_tag}]", f"[{length_tag}]"]
        if has_q: parts.append(has_q)
        if is_show: parts.append(is_show)
        if is_ask: parts.append(is_ask)
        parts.append(title)
        inputs_list.append(" ".join(parts))
    
    # RoBERTa prediction
    inputs = tokenizer(inputs_list, padding=True, truncation=True, 
                       max_length=MAX_LENGTH, return_tensors="pt").to(device)
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        scaled_logits = outputs.logits / optimal_temp
        probs_roberta = torch.softmax(scaled_logits, dim=-1)[:, 1].cpu().numpy()
    
    # TF-IDF prediction
    X_tfidf = tfidf.transform(titles)
    probs_tfidf_pred = tfidf_model.predict_proba(X_tfidf)[:, 1]
    
    # Ensemble
    probs = ROBERTA_WEIGHT * probs_roberta + TFIDF_WEIGHT * probs_tfidf_pred
    
    results = pd.DataFrame({
        'title': titles,
        'domain': domains,
        'probability': probs,
        'prediction': ['HIT' if p >= best_thresh else 'miss' for p in probs]
    })
    return results.sort_values('probability', ascending=False)

# Demo
demo_titles = [
    "Show HN: I built a neural network in Rust",
    "Google announces layoffs", 
    "Python 4.0 released",
    "The decline of Stack Overflow",
    "Ask HN: What's your favorite database?",
    "How we scaled to 1 million users",
    "My weekend project went viral",
]
demo_domains = ["github.com", "reuters.com", "python.org", "other", "self.hackernews", "other", "other"]

print("Demo Predictions (Ensemble):\n")
for _, row in predict_v2_ensemble(demo_titles, demo_domains).iterrows():
    icon = "🔥" if row['prediction'] == 'HIT' else "  "
    print(f"{icon} [{row['probability']:.0%}] {row['title'][:50]}")

## 11. Save Model with Calibration Config

In [ ]:
import pickle

# Save model and tokenizer
model.save_pretrained("./hn_predictor_v2")
tokenizer.save_pretrained("./hn_predictor_v2")

# Save TF-IDF model
with open("./hn_predictor_v2/tfidf_vectorizer.pkl", "wb") as f:
    pickle.dump(tfidf, f)
with open("./hn_predictor_v2/tfidf_model.pkl", "wb") as f:
    pickle.dump(tfidf_model, f)

# Save calibration config with all parameters
config = {
    "model_name": MODEL_NAME,
    "temperature": float(optimal_temp),
    "threshold": float(best_thresh),
    "roberta_weight": ROBERTA_WEIGHT,
    "tfidf_weight": TFIDF_WEIGHT,
    "roc_auc_roberta": float(roc_auc_score(labels, probs_calibrated)),
    "roc_auc_ensemble": float(roc_auc_score(labels, probs_final)),
    "f1_score": float(f1_score(labels, preds_optimal)),
    "ece": float(ece),
    "top_domains": list(top_domains),
    "training_posts": len(df),
    "hit_threshold": HIT_THRESHOLD,
    "max_length": MAX_LENGTH,
}

with open("./hn_predictor_v2/calibration_config.json", "w") as f:
    json.dump(config, f, indent=2)

print("Saved:")
print("  - hn_predictor_v2/model.safetensors")
print("  - hn_predictor_v2/config.json")  
print("  - hn_predictor_v2/tokenizer files")
print("  - hn_predictor_v2/tfidf_vectorizer.pkl")
print("  - hn_predictor_v2/tfidf_model.pkl")
print("  - hn_predictor_v2/calibration_config.json")

In [ ]:
# Create zip for download
!zip -r hn_predictor_v2.zip ./hn_predictor_v2 evaluation_v2.png

print("\n" + "=" * 60)
print("DOWNLOAD: hn_predictor_v2.zip")
print("=" * 60)
print("Find it in the file browser (left panel) and right-click > Download")

## Usage in RSS Reader

```python
import json
import pickle
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

class HNPredictorV2:
    def __init__(self, model_path="./hn_predictor_v2"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_path)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_path)
        self.model.eval()
        
        # Load TF-IDF ensemble
        with open(f"{model_path}/tfidf_vectorizer.pkl", "rb") as f:
            self.tfidf = pickle.load(f)
        with open(f"{model_path}/tfidf_model.pkl", "rb") as f:
            self.tfidf_model = pickle.load(f)
        
        # Load config
        with open(f"{model_path}/calibration_config.json") as f:
            self.config = json.load(f)
        self.top_domains = set(self.config["top_domains"])
    
    def predict(self, titles, domains=None, time_tags=None):
        if domains is None:
            domains = ["other"] * len(titles)
        if time_tags is None:
            time_tags = ["WEEKDAY"] * len(titles)
        
        # Build rich input
        inputs_list = []
        for title, domain, time_tag in zip(titles, domains, time_tags):
            domain_tag = domain if domain in self.top_domains else "other"
            length_tag = "SHORT" if len(title) < 40 else ("MEDIUM" if len(title) < 70 else "LONG")
            parts = [f"[{domain_tag}]", f"[{time_tag}]", f"[{length_tag}]"]
            if "?" in title: parts.append("[Q]")
            if title.lower().startswith("show hn"): parts.append("[SHOW]")
            elif title.lower().startswith("ask hn"): parts.append("[ASK]")
            parts.append(title)
            inputs_list.append(" ".join(parts))
        
        # RoBERTa
        inputs = self.tokenizer(inputs_list, padding=True, truncation=True, 
                                max_length=self.config["max_length"], return_tensors="pt")
        with torch.no_grad():
            logits = self.model(**inputs).logits / self.config["temperature"]
            probs_roberta = torch.softmax(logits, dim=-1)[:, 1].numpy()
        
        # TF-IDF
        probs_tfidf = self.tfidf_model.predict_proba(self.tfidf.transform(titles))[:, 1]
        
        # Ensemble
        probs = self.config["roberta_weight"] * probs_roberta + self.config["tfidf_weight"] * probs_tfidf
        return probs

# Usage
predictor = HNPredictorV2("./hn_predictor_v2")
scores = predictor.predict(
    titles=["Show HN: My new project", "Google layoffs"],
    domains=["github.com", "reuters.com"]
)
```

## All Recommendations Implemented ✓

| Recommendation | Implementation |
|----------------|----------------|
| ✅ Lower threshold | Auto-optimized for best F1 |
| ✅ Domain features | `[domain]` prefix in input |
| ✅ Temporal features | `[PEAK/WEEKDAY/WEEKEND]` tags |
| ✅ TF-IDF ensemble | 70% RoBERTa + 30% TF-IDF |
| ✅ RoBERTa | Upgraded from DistilBERT |
| ✅ Title meta-features | `[SHORT/MEDIUM/LONG]`, `[Q]`, `[SHOW]`, `[ASK]` |